# eWRC API

Simple Python API for eWRC results.

Initially built for pulling results at the end of a rally.

What do we need to do to make it work live too, eg to force refesh on certain stages?

In [8]:
import pandas as pd
import re
from dakar_utils import getTime

In [9]:
import requests
import lxml.html as LH
from bs4 import BeautifulSoup
from bs4.element import NavigableString

## Generic Utilities

Utility functions.

In [10]:
def soupify(url):
    """Load HTML form URL and make into soup."""
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'lxml') # Parse the HTML as a string
    
    # Remove occasional tags that might appear
    # https://stackoverflow.com/a/40760750/454773
    unwanted = soup.find(id="donate-main")
    unwanted.extract()
    
    return soup

In [11]:
def no_children(node):
    """Extract just the text and no child nodes from a soup node."""
    #https://stackoverflow.com/a/31909680/454773
    text = ''.join([t for t in node.contents if type(t) == NavigableString])
    return text

In [12]:
def dfify(table):
    df = pd.read_html('<html><body>{}</body></html>'.format(table))[0]
    df = df.dropna(axis=1, how='all').dropna(axis=0, how='all')
    return df

In [13]:
import unicodedata

In [14]:
def cleanString(s):
    s = unicodedata.normalize("NFKD", str(s))
    #replace multiple whitespace with single space
    s = ' '.join(s.split())
    
    return s

## Timing Utilities

In [15]:
def diffgapsplitter(col):
    #Normalise
    col=col.fillna('+0+0')
    #Remove leading +
    col=col.str.strip('+')
    #Split...
    col = col.str.split('+',expand=True)
    #Rename columns
    col = col.rename(columns={0:'Gap', 1:'Diff'})
    #Convert to numerics
    col['Gap'] = col['Gap'].apply(getTime)#.astype(float)
    col['Diff'] = col['Diff'].apply(getTime)
    return col

## Scraping Functions

In [16]:
base_url = 'https://www.ewrc-results.com'

In [17]:
def get_stage_result_links(stub):
    #If navigation remains constant, items are in third list
    
    rally_stage_results_url='https://www.ewrc-results.com/results/{stub}/'.format(stub=stub)
    
    links={}

    soup = soupify(rally_stage_results_url)
    for li in soup.find_all('ul')[2].find_all('li'):
        #if 'class' in li.attrs:
        #    print(li['class'])
        #A class is set for service but not other things
        if 'class' not in li.attrs:
            a = li.find('a')
            if 'href' in a.attrs:
                #links.append(a['href'])
                links[f'SS{a.text}'] = a['href']
                
    return links

In [18]:
#url='https://www.ewrc-results.com/results/54762-corbeau-seats-rally-tendring-clacton-2019/'
rally_stub = '54762-corbeau-seats-rally-tendring-clacton-2019'
rally_stub='61961-mgj-engineering-brands-hatch-winter-stages-2020'
rally_stub='59972-rallye-automobile-de-monte-carlo-2020'
tmp = get_stage_result_links(rally_stub)
tmp

{'SS1': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241659',
 'SS2': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241660',
 'SS3': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241661',
 'SS4': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241662',
 'SS5': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241663',
 'SS6': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241664',
 'SS7': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241665',
 'SS8': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241666',
 'SS2. leg': '/leg/59972-rallye-automobile-de-monte-carlo-2020/?leg=2',
 'SS9': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241667',
 'SS10': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241668',
 'SS11': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241669',
 'SS12': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241670',
 'SS3. leg': '/leg/59972-rallye-autom

In [19]:
stage_result_cols = ['Pos', 'CarNum', 'Desc', 'Class', 'Time', 'GapDiff', 'Speedkm', 'Stage',
       'StageName', 'StageDist', 'Gap', 'Diff', 'Speed', 'Dist', 'entryId',
       'model', 'navigator', 'PosNum']

In [20]:
stage_overall_cols = ['PosChange', 'CarNum', 'Desc', 'Class', 'Time', 'GapDiff', 'Speedkm',
       'Stage', 'StageName', 'StageDist', 'Pos', 'Change', 'Gap', 'Diff',
       'Speed', 'Dist']

In [21]:
retirement_cols = ['CarNum', 'driverNav', 'Model', 'Status']
retirement_extra_cols = ['Driver', 'CoDriver', 'Stage']

In [22]:
penalty_cols = ['CarNum', 'driverNav', 'Model', 'PenReason']
penalty_extra_cols = ['Driver', 'CoDriver', 'Stage', 'Time','Reason']

In [23]:
from numpy import nan

In [24]:
from parse import parse   

In [25]:
details = 'SS16 La Cabanette - Col de Braus 2 [Power Stage] - 13.36 km - 26. 1. 12:18'
#details = 'SS6 Curbans - Venterol 2 - 20.02 km - 24. 1. 13:54'
pattern = 'SS{stage} {name} - {dist:f} km - {datetime}'
parse(pattern, details)

<Result () {'stage': '16', 'name': 'La Cabanette - Col de Braus 2 [Power Stage]', 'dist': 13.36, 'datetime': '26. 1. 12:18'}>

In [26]:
def get_stage_results(stub):
    soup = soupify('{}{}'.format(base_url, stub))

    details = soup.find('h4').text

    pattern = 'SS{stage} {name} - {dist:f} km - {datetime}'
    parse_result = parse(pattern, details)
    if parse_result is None:
        pattern = 'SS{stage} - {dist:f} km'
        parse_result = parse(pattern, details)
    print(details, parse_result)
    stage_num = f"SS{parse_result['stage']}"
  
    if 'name' in parse_result:
        stage_name = parse_result['name']
    else:
         stage_name = stage_num
    
    stage_dist =  parse_result['dist']
    if 'datetime' in parse_result:
        stage_datetime = parse_result['datetime']
    else:
        stage_datetime = None
    
    tables = soup.find_all('table')
    stage_result = tables[0]
    stage_overall = tables[1]
    
    # Stage Result
    df_stage_result = dfify(stage_result)
    df_stage_result.columns=['Pos','CarNum','Desc','Class', 'Time','GapDiff', 'Speedkm']
    
    df_stage_result['Stage'] = stage_num
    df_stage_result['StageName'] = stage_name
    df_stage_result['StageDist'] = stage_dist
    
    df_stage_result['GapDiff'].fillna('+0+0').str.strip('+').str.split('+',expand=True).rename(columns={0:'Gap', 1:'Diff'})
    df_stage_result[['Gap','Diff']] = diffgapsplitter(df_stage_result['GapDiff'])
    df_stage_result[['Speed','Dist']] = df_stage_result['Speedkm'].str.extract(r'(?P<Speed>[^.]*\.[\d])(?P<Dist>.*)')
    
    rows=[]
    for d in stage_result.findAll("td", {"class": "stage-results-drivers"}):
        entryId = d.find('a')['href']
        #print(str(d)) #This gives us the raw HTML in the soup element
        driverNav = d.find('a').text.split('-')
        model=d.find('a').nextSibling.nextSibling
        rows.append( {'entryId':entryId,
                       'model':model,
                      'driver':cleanString(driverNav[0]),
                      'navigator':cleanString(driverNav[1])}) 

    df_stage_result[['driver','entryId','model','navigator']] = pd.DataFrame(rows)
    #Should we cast the Pos to a numeric too? Set = to na then ffill down?
    df_stage_result['PosNum'] = df_stage_result['Pos'].replace('=',nan).astype(float).fillna(method='ffill').astype(int)
    df_stage_result.set_index('driver',drop=True, inplace=True)
    
    # Stage Overall
    df_stage_overall = dfify(stage_overall)

    cols = ['PosChange', 'CarNum', 'Desc','Class', 'Time', 'GapDiff', 'Speedkm' ]
    df_stage_overall.columns = cols

    df_stage_overall['Stage'] = stage_num
    df_stage_overall['StageName'] = stage_name
    df_stage_overall['StageDist'] = stage_dist
    
    df_stage_overall[['Pos','Change']] = df_stage_overall['PosChange'].astype(str).str.extract(r'(?P<Pos>[\d]*)\.\s?(?P<Change>.*)?')
    df_stage_overall['GapDiff'].fillna('+0+0').str.strip('+').str.split('+',expand=True).rename(columns={0:'Gap', 1:'Diff'})
    df_stage_overall[['Gap','Diff']] = diffgapsplitter(df_stage_overall['GapDiff'])
    df_stage_overall[['Speed','Dist']] = df_stage_overall['Speedkm'].str.extract(r'(?P<Speed>[^.]*\.[\d])(?P<Dist>.*)')

    
    # Retirements
    df_stage_retirements = pd.DataFrame(columns=retirement_cols+retirement_extra_cols)
    retired = soup.find('div',{'class':'retired-inc'})
    if retired:
        df_stage_retirements = dfify(retired.find('table'))
        df_stage_retirements.columns = retirement_cols
        df_stage_retirements[['Driver','CoDriver']] = df_stage_retirements['driverNav'].str.extract(r'(?P<Driver>.*)\s+-\s+(?P<CoDriver>.*)')
        df_stage_retirements['Stage'] = stage_num
    
    # Penalties
    df_stage_penalties = pd.DataFrame(columns=penalty_cols+penalty_extra_cols)

    penalty = soup.find('div',{'class':'penalty-inc'})
    if penalty:
        df_stage_penalties = dfify(penalty.find('table'))
        df_stage_penalties.columns = penalty_cols
        df_stage_penalties[['Driver','CoDriver']] = df_stage_penalties['driverNav'].str.extract(r'(?P<Driver>.*)\s+-\s+(?P<CoDriver>.*)')
        df_stage_penalties[['Time','Reason']] = df_stage_penalties['PenReason'].str.extract(r'(?P<Time>[^\s]*)\s+(?P<Reason>.*)')
        df_stage_penalties['Stage'] = stage_num
    
    return df_stage_result, df_stage_overall, df_stage_retirements, df_stage_penalties

In [27]:
partial_stub = '/results/54762-corbeau-seats-rally-tendring-clacton-2019/'
partial_stub='/results/42870-rallye-automobile-de-monte-carlo-2018/'
partial_stub='/results/61961-mgj-engineering-brands-hatch-winter-stages-2020/'
partial_stub='/results/59972-rallye-automobile-de-monte-carlo-2020/'
#stub = tmp['SS3']
stage_result, stage_overall, stage_retirements, stage_penalties = get_stage_results(partial_stub)

SS16 La Cabanette - Col de Braus 2 [Power Stage] - 13.36 km - 26. 1. 12:18 <Result () {'stage': '16', 'name': 'La Cabanette - Col de Braus 2 [Power Stage]', 'dist': 13.36, 'datetime': '26. 1. 12:18'}>


In [28]:
stage_result.head()

,Pos,CarNum,Desc,Class,Time,GapDiff,Speedkm,Stage,StageName,StageDist,Gap,Diff,Speed,Dist,entryId,model,navigator,PosNum
driver,,,,,,,,,,,,,,,,,,
/entryinfo/59972-rallye-automobile-de-monte-carlo-2020/2465681/,1.,11,Neuville Thierry - Gilsoul N.Hyundai i20 Coupe...,RC1M,9:39.0,NaN,83.1,SS16,La Cabanette - Col de Braus 2 [Power Stage],13.36,0.0,0.0,83.1,,Hyundai i20 Coupe WRC,Neuville Thierry,Gilsoul N.,1
/entryinfo/59972-rallye-automobile-de-monte-carlo-2020/2465684/,=,17,Ogier Sébastien - Ingrassia J.Toyota Yaris WRC,RC1M,9:39.0,+0.0+0.0,83.10.00,SS16,La Cabanette - Col de Braus 2 [Power Stage],13.36,0.0,0.0,83.1,0.00,Toyota Yaris WRC,Ogier Sébastien,Ingrassia J.,1
/entryinfo/59972-rallye-automobile-de-monte-carlo-2020/2487704/,3.,3,Suninen Teemu - Lehtinen JarmoFord Fiesta WRC,RC1M,9:41.1,+2.1+2.1,82.80.16,SS16,La Cabanette - Col de Braus 2 [Power Stage],13.36,2.1,2.1,82.8,0.16,Ford Fiesta WRC,Suninen Teemu,Lehtinen Jarmo,3
/entryinfo/59972-rallye-automobile-de-monte-carlo-2020/2465687/,4.,33,Evans Elfyn - Martin ScottToyota Yaris WRC,RC1M,9:42.2,+3.2+1.1,82.60.24,SS16,La Cabanette - Col de Braus 2 [Power Stage],13.36,3.2,1.1,82.6,0.24,Toyota Yaris WRC,Evans Elfyn,Martin Scott,4
/entryinfo/59972-rallye-automobile-de-monte-carlo-2020/2487703/,5.,4,Lappi Esapekka - Ferm JanneFord Fiesta WRC,RC1M,9:45.2,+6.2+3.0,82.20.46,SS16,La Cabanette - Col de Braus 2 [Power Stage],13.36,6.2,3.0,82.2,0.46,Ford Fiesta WRC,Lappi Esapekka,Ferm Janne,5


In [29]:
stage_overall

,PosChange,CarNum,Desc,Class,Time,GapDiff,Speedkm,Stage,StageName,StageDist,Pos,Change,Gap,Diff,Speed,Dist
0,1.,11,Neuville T. - Gilsoul N.Hyundai i20 Coupe WRC,RC1M,3:10:57.6,NaN,95.6,SS16,La Cabanette - Col de Braus 2 [Power Stage],13.36,1,,0.0,0.0,95.6,
1,2. +1,17,Ogier S. - Ingrassia J.Toyota Yaris WRC,RC1M,3:11:10.2,+12.6+12.6,95.50.04,SS16,La Cabanette - Col de Braus 2 [Power Stage],13.36,2,+1,12.6,12.6,95.5,0.04
2,3. −1,33,Evans Elfyn - Martin ScottToyota Yaris WRC,RC1M,3:11:11.9,+14.3+1.7,95.50.05,SS16,La Cabanette - Col de Braus 2 [Power Stage],13.36,3,−1,14.3,1.7,95.5,0.05
3,4.,4,Lappi Esapekka - Ferm JanneFord Fiesta WRC,RC1M,3:14:06.6,+3:09.0+2:54.7,94.10.62,SS16,La Cabanette - Col de Braus 2 [Power Stage],13.36,4,,189.0,174.7,94.1,0.62
4,5.,69,Rovanperä K. - Halttunen J.Toyota Yaris WRC,RC1M,3:15:14.8,+4:17.2+1:08.2,93.50.85,SS16,La Cabanette - Col de Braus 2 [Power Stage],13.36,5,,257.2,68.2,93.5,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,69.,84,Cartagena J. - Aubery A.Peugeot 208 R2,RC4,4:51:30.02:00,+1:40:32.4+31.2,63.119.8,SS16,La Cabanette - Col de Braus 2 [Power Stage],13.36,69,,6032.4,31.2,63.1,19.8
69,70.,92,Jerusalmi J. - Luthen G.Renault Twingo RS R1,RC5,4:51:52.40:30,+1:40:54.8+22.4,62.719.9,SS16,La Cabanette - Col de Braus 2 [Power Stage],13.36,70,,6054.8,22.4,62.7,19.9
70,71.,25,Nobre Paulo - Morales G.Škoda Fabia R5 [SR],RC2,4:54:40.7,+1:43:43.1+2:48.3,62.020.5,SS16,La Cabanette - Col de Braus 2 [Power Stage],13.36,71,,6223.1,168.3,62.0,20.5
71,72.,59,Dizier Grégory - Dizier S.Renault Clio RS R3T ...,RC3,5:03:38.80:10,+1:52:41.2+8:58.1,60.222.2,SS16,La Cabanette - Col de Braus 2 [Power Stage],13.36,72,,6761.2,538.1,60.2,22.2


In [30]:
stage_retirements

,CarNum,driverNav,Model,Status,Driver,CoDriver,Stage


In [31]:
stage_penalties

,CarNum,driverNav,Model,PenReason,Driver,CoDriver,Time,Reason,Stage
0,21,Gryazin Nikolay - Fedorov Y.,Hyundai i20 R5,0:20 Late (2 min) at TC,Gryazin Nikolay,Fedorov Y.,0:20,Late (2 min) at TC,SS16
1,85,Cartier V. - Margaillan M.,Ford Fiesta R2T,0:20 Late (2 min) at TC,Cartier V.,Margaillan M.,0:20,Late (2 min) at TC,SS16
2,70,Ribaudo Tony - Degrange J.,Peugeot 208 R2,0:10 Late (1 min) at TC,Ribaudo Tony,Degrange J.,0:10,Late (1 min) at TC,SS16


In [32]:
from parse import parse

In [33]:
def check_time_str(txt):
    """Clean a time string."""
    
    # Quick fix to cope with strings of the form:
    # '11:58.0 <a name="" title="Notional time"><span class="c-blue">[N]</span></a>'
    # This should be a proper validator.
    txt = txt.split()[0]
    return txt

In [60]:
def get_stage_times(stub, dropnarow=True):
    url='https://www.ewrc-results.com/times/{stub}/'.format(stub=stub)

    soup = soupify(url)
    
    times = soup.find('div',{'class':'times'}).findChildren('div' , recursive=False)

    #The rows are essentially grouped in twos after the header row
    cols = [c.text for c in times[0].findAll('div')]
    
    groupsize=2
    groups = [times[i:i+groupsize] for i in range(1, len(times), groupsize)]
    
    NAME_SUBGROUP = 0
    TIME_SUBGROUP = 1
    
    carNumMatch = lambda txt: re.search('#(?P<carNum>[\d]*)', cleanString(txt))
    carModelMatch = lambda txt:  re.search('</a>\s*(?P<carModel>.*)</div>', cleanString(txt))
    
    pattern = '''<div class="times-one-time">{stagetime}<br/><span class="times-after">{overalltime}</span><br/>{pos}</div>'''
    
    t=[]
    i=0

    penaltypattern='class="r7_bold_red">{penalty}</span>'

    for g in groups:
        i=i+1
        driverNav_el = g[NAME_SUBGROUP].find('a')
        driverNav = driverNav_el.text
        driver,navigator = driverNav.split(' - ')
        entryId = driverNav_el['href']
        retired = '<span class="r8_bold_red">R</span>' in str(g[NAME_SUBGROUP])
        carNum = carNumMatch(g[NAME_SUBGROUP]).group('carNum')
        carModel = carModelMatch(g[NAME_SUBGROUP]).group('carModel')
        
        #TO DO - may be None?
        try:
            classification = pd.to_numeric(g[NAME_SUBGROUP].find('span').text.replace('R','').strip('').strip('.'))
        except:
            classification = ''
        
        stagetimes = []
        overalltimes = []
        penalties=[]
        positions = []

        for stages in g[TIME_SUBGROUP].findAll('div'):
            txt = cleanString(stages)
            stagetimes_data = parse(pattern, txt )
            if stagetimes_data:
                stagetimes.append(check_time_str(stagetimes_data['stagetime']))
                overalltimes.append(check_time_str(stagetimes_data['overalltime']))

                #Need to parse this
                #There may be penalties in the pos
                penalty = 0
                p = stagetimes_data['pos'].split()
                if p[-1].endswith('</span>'):
                    penalty = parse(penaltypattern, p[-1] )
                    if penalty:
                        #This really needs parsing into a time; currently of form eg 0:10
                        penalty = penalty['penalty']
                    # TO DO - what if we have a replacement? Error codes?
                    if '>R<' in p[-1]:
                        p = 0
                    else:
                        p = int(p[-2].split('.')[0])
                else:
                    p = int(p[-1].strip('.'))
                positions.append(p)
                penalties.append(penalty)
            # TO DO - how do we account for cancelled stages?
            # If we add in blanks we get balnks for un-run stages too?
            #else:
            #    stagetimes.append('')
            #    overalltimes.append('')
             #   positions.append('')
            #    penalties.append('')

        t.append({'entryId': entryId,
                  'driverNav': driverNav,
                  'driver': driver.strip(),
                  'navigator': navigator.strip(),
                  'carNum': carNum,
                  'carModel': carModel,
                  'retired': retired,
                  'Pos': classification,
                  'stagetimes': stagetimes,
                  'overalltimes': overalltimes,
                  'positions': positions,
                  'penalties': penalties})


    df_allInOne = pd.DataFrame(t).set_index(['entryId'])
    
    df_overall = pd.DataFrame(df_allInOne['overalltimes'].tolist(), index= df_allInOne.index)
    df_overall.columns = range(1, df_overall.shape[1]+1)
    
    df_overall_pos = pd.DataFrame(df_allInOne['positions'].tolist(), index= df_allInOne.index)
    df_overall_pos.columns = range(1, df_overall_pos.shape[1]+1)

    df_stages = pd.DataFrame(df_allInOne['stagetimes'].tolist(), index= df_allInOne.index)
    df_stages.columns = range(1, df_stages.shape[1]+1)
    
    df_stages_pos = df_stages.reset_index().drop_duplicates(subset='entryId').set_index('entryId').rank(method='min')
    df_stages_pos.columns = range(1, df_stages_pos.shape[1]+1)

    xcols = df_overall.columns

    for ss in xcols:
        df_overall[ss] = df_overall[ss].apply(getTime)
        df_stages[ss] = df_stages[ss].apply(getTime)

    # TO DO
    #We shouldn't really have to do this - why are there duplicates?
    #We seem to be appending rows over and over for each stage?
    df_allInOne = df_allInOne.reset_index().drop_duplicates(subset='entryId').set_index('entryId')
    df_overall = df_overall.reset_index().drop_duplicates(subset='entryId').set_index('entryId')
    df_stages = df_stages.reset_index().drop_duplicates(subset='entryId').set_index('entryId')
    df_overall_pos = df_overall_pos.reset_index().drop_duplicates(subset='entryId').set_index('entryId')
    
    if dropnarow:
        df_allInOne = df_allInOne.dropna(how='all', axis=1)
        df_overall = df_overall.dropna(how='all', axis=1)
        df_stages = df_stages.dropna(how='all', axis=1)
        df_overall_pos = df_overall_pos.dropna(how='all', axis=1)
        df_stages_pos = df_stages_pos.dropna(how='all', axis=1)
        
    return df_allInOne, df_overall, df_stages, df_overall_pos, df_stages_pos

In [61]:
url='https://www.ewrc-results.com/times/54762-corbeau-seats-rally-tendring-clacton-2019/'
#url='https://www.ewrc-results.com/times/42870-rallye-automobile-de-monte-carlo-2018/'

In [62]:
rally_stub = '42870-rallye-automobile-de-monte-carlo-2018'
rally_stub='61961-mgj-engineering-brands-hatch-winter-stages-2020'
rally_stub='59972-rallye-automobile-de-monte-carlo-2020'
rally_stub='60140-rally-sweden-2020'

df_allInOne, df_overall, df_stages, \
    df_overall_pos, df_stages_pos = get_stage_times(rally_stub)

In [63]:
display(df_allInOne)
display(df_overall)
display(df_stages)
display(df_overall_pos)
display(df_stages_pos)

,driverNav,driver,navigator,carNum,carModel,retired,Pos,stagetimes,overalltimes,positions,penalties
entryId,,,,,,,,,,,
/entryinfo/60140-rally-sweden-2020/2496932/,Evans Elfyn - Martin Scott,Evans Elfyn,Martin Scott,33,Toyota Yaris WRC,False,,"[9:43.9, 10:14.3, 9:02.9, 1:42.6, 9:25.2, 9:53...","[9:43.9, 19:58.2, 29:01.1, 30:43.7, 40:08.9, 5...","[1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0]"
/entryinfo/60140-rally-sweden-2020/2494761/,Tänak Ott - Järveoja Martin,Tänak Ott,Järveoja Martin,8,Hyundai i20 Coupe WRC,False,,"[9:45.0, 10:13.4, 9:11.4, 1:42.4, 9:28.4, 9:57...","[9:45.0, 19:58.4, 29:09.8, 30:52.2, 40:20.6, 5...","[2, 2, 3, 2, 2, 2, 2]","[0, 0, 0, 0, 0, 0, 0]"
/entryinfo/60140-rally-sweden-2020/2496933/,Rovanperä Kalle - Halttunen Jonne,Rovanperä Kalle,Halttunen Jonne,69,Toyota Yaris WRC,False,,"[9:45.9, 10:17.6, 9:05.5, 1:49.0, 9:32.1, 9:59...","[9:45.9, 20:03.5, 29:09.0, 30:58.0, 40:30.1, 5...","[3, 3, 2, 3, 3, 4, 3]","[0, 0, 0, 0, 0, 0, 0]"
/entryinfo/60140-rally-sweden-2020/2496931/,Ogier Sébastien - Ingrassia Julien,Ogier Sébastien,Ingrassia Julien,17,Toyota Yaris WRC,False,,"[9:46.8, 10:19.6, 9:11.1, 1:44.0, 9:28.7, 9:57...","[9:46.8, 20:06.4, 29:17.5, 31:01.5, 40:30.2, 5...","[4, 4, 4, 4, 4, 3, 4]","[0, 0, 0, 0, 0, 0, 0]"
/entryinfo/60140-rally-sweden-2020/2501811/,Lappi Esapekka - Ferm Janne,Lappi Esapekka,Ferm Janne,4,Ford Fiesta WRC,False,,"[9:50.2, 10:19.7, 9:11.4, 1:43.3, 9:32.5, 9:58...","[9:50.2, 20:09.9, 29:21.3, 31:04.6, 40:37.1, 5...","[6, 5, 5, 5, 5, 5, 5]","[0, 0, 0, 0, 0, 0, 0]"
/entryinfo/60140-rally-sweden-2020/2494762/,Neuville Thierry - Gilsoul Nicolas,Neuville Thierry,Gilsoul Nicolas,11,Hyundai i20 Coupe WRC,False,,"[9:48.3, 10:21.8, 9:13.0, 1:44.2, 9:31.0, 9:58...","[9:48.3, 20:10.1, 29:23.1, 31:07.3, 40:38.3, 5...","[5, 6, 6, 6, 6, 6, 6]","[0, 0, 0, 0, 0, 0, 0]"
/entryinfo/60140-rally-sweden-2020/2494764/,Breen Craig - Nagle Paul,Breen Craig,Nagle Paul,16,Hyundai i20 Coupe WRC,False,,"[9:53.3, 10:19.8, 9:11.8, 1:43.0, 9:37.2, 9:59...","[9:53.3, 20:13.1, 29:24.9, 31:07.9, 40:45.1, 5...","[8, 7, 7, 7, 7, 7, 7]","[0, 0, 0, 0, 0, 0, 0]"
/entryinfo/60140-rally-sweden-2020/2501812/,Suninen Teemu - Lehtinen Jarmo,Suninen Teemu,Lehtinen Jarmo,3,Ford Fiesta WRC,False,,"[9:52.1, 10:26.2, 9:12.9, 1:43.9, 9:40.2, 10:0...","[9:52.1, 20:18.3, 29:31.2, 31:15.1, 40:55.3, 5...","[7, 8, 8, 8, 8, 8, 8]","[0, 0, 0, 0, 0, 0, 0]"
/entryinfo/60140-rally-sweden-2020/2496941/,Katsuta Takamoto - Barritt Daniel,Katsuta Takamoto,Barritt Daniel,18,Toyota Yaris WRC,False,,"[9:57.0, 10:28.9, 9:20.4, 1:47.0, 9:42.2, 10:0...","[9:57.0, 20:25.9, 29:46.3, 31:33.3, 41:15.5, 5...","[10, 9, 9, 9, 9, 9, 9]","[0, 0, 0, 0, 0, 0, 0]"


,1,2,3,4,5,6,7
entryId,,,,,,,
/entryinfo/60140-rally-sweden-2020/2496932/,583.9,1198.2,1741.1,1843.7,2408.9,3002.6,3535.7
/entryinfo/60140-rally-sweden-2020/2494761/,585.0,1198.4,1749.8,1852.2,2420.6,3017.9,3552.6
/entryinfo/60140-rally-sweden-2020/2496933/,585.9,1203.5,1749.0,1858.0,2430.1,3029.6,3563.7
/entryinfo/60140-rally-sweden-2020/2496931/,586.8,1206.4,1757.5,1861.5,2430.2,3027.8,3565.0
/entryinfo/60140-rally-sweden-2020/2501811/,590.2,1209.9,1761.3,1864.6,2437.1,3035.5,3570.8
/entryinfo/60140-rally-sweden-2020/2494762/,588.3,1210.1,1763.1,1867.3,2438.3,3036.3,3575.7
/entryinfo/60140-rally-sweden-2020/2494764/,593.3,1213.1,1764.9,1867.9,2445.1,3044.4,3583.8
/entryinfo/60140-rally-sweden-2020/2501812/,592.1,1218.3,1771.2,1875.1,2455.3,3059.4,3609.7
/entryinfo/60140-rally-sweden-2020/2496941/,597.0,1225.9,1786.3,1893.3,2475.5,3083.0,3633.0


,1,2,3,4,5,6,7
entryId,,,,,,,
/entryinfo/60140-rally-sweden-2020/2496932/,583.9,614.3,542.9,102.6,565.2,593.7,533.1
/entryinfo/60140-rally-sweden-2020/2494761/,585.0,613.4,551.4,102.4,568.4,597.3,534.7
/entryinfo/60140-rally-sweden-2020/2496933/,585.9,617.6,545.5,109.0,572.1,599.5,534.1
/entryinfo/60140-rally-sweden-2020/2496931/,586.8,619.6,551.1,104.0,568.7,597.6,537.2
/entryinfo/60140-rally-sweden-2020/2501811/,590.2,619.7,551.4,103.3,572.5,598.4,535.3
/entryinfo/60140-rally-sweden-2020/2494762/,588.3,621.8,553.0,104.2,571.0,598.0,539.4
/entryinfo/60140-rally-sweden-2020/2494764/,593.3,619.8,551.8,103.0,577.2,599.3,539.4
/entryinfo/60140-rally-sweden-2020/2501812/,592.1,626.2,552.9,103.9,580.2,604.1,550.3
/entryinfo/60140-rally-sweden-2020/2496941/,597.0,628.9,560.4,107.0,582.2,607.5,550.0


,1,2,3,4,5,6,7
entryId,,,,,,,
/entryinfo/60140-rally-sweden-2020/2496932/,1,1.0,1.0,1.0,1.0,1.0,1.0
/entryinfo/60140-rally-sweden-2020/2494761/,2,2.0,3.0,2.0,2.0,2.0,2.0
/entryinfo/60140-rally-sweden-2020/2496933/,3,3.0,2.0,3.0,3.0,4.0,3.0
/entryinfo/60140-rally-sweden-2020/2496931/,4,4.0,4.0,4.0,4.0,3.0,4.0
/entryinfo/60140-rally-sweden-2020/2501811/,6,5.0,5.0,5.0,5.0,5.0,5.0
/entryinfo/60140-rally-sweden-2020/2494762/,5,6.0,6.0,6.0,6.0,6.0,6.0
/entryinfo/60140-rally-sweden-2020/2494764/,8,7.0,7.0,7.0,7.0,7.0,7.0
/entryinfo/60140-rally-sweden-2020/2501812/,7,8.0,8.0,8.0,8.0,8.0,8.0
/entryinfo/60140-rally-sweden-2020/2496941/,10,9.0,9.0,9.0,9.0,9.0,9.0


,1,2,3,4,5,6,7
entryId,,,,,,,
/entryinfo/60140-rally-sweden-2020/2496932/,37.0,2.0,27.0,3.0,35.0,36.0,23.0
/entryinfo/60140-rally-sweden-2020/2494761/,38.0,1.0,30.0,2.0,36.0,37.0,25.0
/entryinfo/60140-rally-sweden-2020/2496933/,39.0,3.0,28.0,18.0,39.0,42.0,24.0
/entryinfo/60140-rally-sweden-2020/2496931/,40.0,4.0,29.0,7.0,37.0,38.0,27.0
/entryinfo/60140-rally-sweden-2020/2501811/,42.0,5.0,30.0,5.0,40.0,40.0,26.0
/entryinfo/60140-rally-sweden-2020/2494762/,41.0,7.0,34.0,8.0,38.0,39.0,28.0
/entryinfo/60140-rally-sweden-2020/2494764/,44.0,6.0,32.0,4.0,41.0,41.0,28.0
/entryinfo/60140-rally-sweden-2020/2501812/,43.0,8.0,33.0,6.0,42.0,1.0,31.0
/entryinfo/60140-rally-sweden-2020/2496941/,46.0,9.0,35.0,11.0,43.0,2.0,30.0


## Final Results

Get overall rankings.

In [107]:
final_path = 'https://www.ewrc-results.com/final/{stub}/'

In [108]:
def get_final(stub):
    soup = soupify(final_path.format(stub=stub))
    tables = soup.find_all('table')
    #tables = LH.fromstring(html).xpath('//table')
    df_rally_overall = pd.read_html('<html><body>{}</body></html>'.format(tables[0]))[0]
    #df_rally_overall['badge'] = [img.find('img')['src'] for img in tables[0].findAll("td", {"class": "final-results-icon"}) ]
    df_rally_overall.dropna(how='all', axis=1, inplace=True)
    df_rally_overall.columns=['Pos','CarNum','driverNav','ModelReg','Class', 'Time','GapDiff', 'Speedkm']

    #Get the entry ID - use this as the unique key
    #in column 3, <a title='Entry info and stats'>
    df_rally_overall['entryId']=[a['href'] for a in tables[0].findAll("a", {"title": "Entry info and stats"}) ]
    df_rally_overall.set_index('entryId', inplace=True)

    df_rally_overall[['Driver','CoDriver']] = df_rally_overall['driverNav'].str.extract(r'(?P<Driver>.*)\s+-\s+(?P<CoDriver>.*)')

    df_rally_overall['Historic']= df_rally_overall['Class'].str.contains('Historic')
    df_rally_overall['Class']= df_rally_overall['Class'].str.replace('Historic','')

    df_rally_overall['Pos'] = df_rally_overall['Pos'].astype(str).str.extract(r'(.*)\.')
    df_rally_overall['Pos'] = df_rally_overall['Pos'].astype(int)

    df_rally_overall[['Model','Registration']]=df_rally_overall['ModelReg'].str.extract(r'(?P<Model>.*) \((?P<Registration>.*)\)')

    df_rally_overall["Class Rank"] = df_rally_overall.groupby("Class")["Pos"].rank(method='min')

    return df_rally_overall
    


In [196]:
get_final(rally_stub)

ValueError: No tables found matching pattern '.+'

## Itinerary

In [109]:
itinerary_path = 'https://www.ewrc-results.com/timetable/{stub}/'

In [110]:
def get_itinerary(stub):
    soup = soupify(itinerary_path.format(stub=stub))
    
    event_dist = soup.find('td',text='Event total').parent.find_all('td')[-1].text

    itinerary_df = dfify( soup.find('div', {'class':'timetable'}).find('table') )
    itinerary_df.columns = ['Stage','Name', 'distance', 'Date', 'Time']
    itinerary_df['Leg'] = [nan if 'leg' not in str(x) else str(x).replace('. leg','') for x in itinerary_df['Stage']]
    itinerary_df['Leg'] = itinerary_df['Leg'].fillna(method='ffill')
    itinerary_df['Date'] = itinerary_df['Date'].fillna(method='ffill')
    
    #What if we have no stage name?
    itinerary_df['Name'].fillna('', inplace=True)
    
    itinerary_leg_totals = itinerary_df[itinerary_df['Name'].str.contains("Leg total")][['Leg', 'distance']].reset_index(drop=True)

    full_itinerary_df = itinerary_df[~itinerary_df['Name'].str.contains(". leg")]
    full_itinerary_df = full_itinerary_df[~full_itinerary_df['Date'].str.contains(" km")]
    full_itinerary_df = full_itinerary_df.fillna(method='bfill', axis=1)

    #Legs may not be identified but we may want to identify services
    full_itinerary_df['Service'] = [ 'Service' in i for i in full_itinerary_df['distance'] ]
    full_itinerary_df['Service_Num'] = full_itinerary_df['Service'].cumsum()
    full_itinerary_df.reset_index(drop=True, inplace=True)
    #itinerary_df = full_itinerary_df[~full_itinerary_df['Service']].reset_index(drop=True)
    itinerary_df = full_itinerary_df[full_itinerary_df['Stage'].str.startswith('SS')].reset_index(drop=True)
    itinerary_df['Section'] = itinerary_df['Service_Num'].rank(method='dense')
    itinerary_df.drop(columns=['Service', 'Service_Num'], inplace=True)
    
    itinerary_df[['Distance', 'Distance_unit']] = itinerary_df['distance'].str.extract(r'(?P<Distance>[^\s]*)\s+(?P<Distance_unit>.*)?')
    itinerary_df['Distance'] = itinerary_df['Distance'].astype(float)

    itinerary_df.set_index('Stage', inplace=True)

    return event_dist, itinerary_leg_totals, itinerary_df, full_itinerary_df

In [111]:
stub='54762-corbeau-seats-rally-tendring-clacton-2019'
stub='42870-rallye-automobile-de-monte-carlo-2018'
#stub='61961-mgj-engineering-brands-hatch-winter-stages-2020'
stub='59972-rallye-automobile-de-monte-carlo-2020'
event_dist, itinerary_leg_totals, itinerary_df, full_itinerary_df = get_itinerary(stub)

In [112]:
print(event_dist)
display(itinerary_leg_totals)
display(itinerary_df)
display(full_itinerary_df)

304.28 km


,Leg,distance
0,1,42.96 km
1,2,122.58 km
2,3,75.20 km
3,4,63.54 km


,Name,distance,Date,Time,Leg,Section,Distance,Distance_unit
Stage,,,,,,,,
SS1,Malijai - Puimichel,17.47 km,23. 1.,20:38,1,1.0,17.47,km
SS2,Bayons - Bréziers,25.49 km,23. 1.,22:26,1,1.0,25.49,km
SS3,Curbans - Venterol 1,20.02 km,24. 1.,08:36,2,2.0,20.02,km
SS4,St-Clément-sur-Durance - Freissinières 1,20.68 km,24. 1.,09:56,2,2.0,20.68,km
SS5,Avançon - Notre-Dame-du-Laus 1,20.59 km,24. 1.,11:21,2,2.0,20.59,km
SS6,Curbans - Venterol 2,20.02 km,24. 1.,13:54,2,3.0,20.02,km
SS7,St-Clément-sur-Durance - Freissinières 2,20.68 km,24. 1.,15:14,2,3.0,20.68,km
SS8,Avançon - Notre-Dame-du-Laus 2,20.59 km,24. 1.,16:39,2,3.0,20.59,km
SS9,St-Léger-les-Mélèzes - La Bâtie-Neuve 1,16.87 km,25. 1.,09:38,3,4.0,16.87,km


,Stage,Name,distance,Date,Time,Leg,Service,Service_Num
0,Shakedown (Gap),Shakedown (Gap),3.35 km,22. 1.,16:01,NaN,False,0
1,SS1,Malijai - Puimichel,17.47 km,23. 1.,20:38,1,False,0
2,SS2,Bayons - Bréziers,25.49 km,23. 1.,22:26,1,False,0
3,Flexi Service A - Gap - 45+3 min,Flexi Service A - Gap - 45+3 min,Flexi Service A - Gap - 45+3 min,23. 1.,23:36,1,True,1
4,Service B - Gap - 15+3 min,Service B - Gap - 15+3 min,Service B - Gap - 15+3 min,24. 1.,07:45,2,True,2
5,SS3,Curbans - Venterol 1,20.02 km,24. 1.,08:36,2,False,2
6,SS4,St-Clément-sur-Durance - Freissinières 1,20.68 km,24. 1.,09:56,2,False,2
7,SS5,Avançon - Notre-Dame-du-Laus 1,20.59 km,24. 1.,11:21,2,False,2
8,Service C - Gap - 40+3 min,Service C - Gap - 40+3 min,Service C - Gap - 40+3 min,24. 1.,12:38,2,True,3
9,SS6,Curbans - Venterol 2,20.02 km,24. 1.,13:54,2,False,3


In [113]:
_tmp = itinerary_df['Distance']
_tmp.index = [int(i.lstrip('SS')) for i in _tmp.index]
_tmp

1     17.47
2     25.49
3     20.02
4     20.68
5     20.59
6     20.02
7     20.68
8     20.59
9     16.87
10    20.73
11    16.87
12    20.73
13    18.41
14    13.36
15    18.41
16    13.36
Name: Distance, dtype: float64

## Entry List

Get the entry list.

In [114]:
entrylist_path = "https://www.ewrc-results.com/entries/{stub}/"

In [115]:
def get_entry_list(stub):
    entrylist_url = entrylist_path.format(stub=stub)
    
    soup = soupify(entrylist_url)
    entrylist_table = soup.find('div',{'class':'startlist'}).find('table')
    df_entrylist = dfify(entrylist_table)
    
    base_cols = ['CarNum', 'DriverName','CoDriverName','Team','Car','Class']
    for i in range(len(df_entrylist.columns) - len(base_cols)):
        base_cols.append(f'Meta_{i}')
    df_entrylist.columns = base_cols
    df_entrylist['carNum'] = df_entrylist['CarNum'].str.extract(r'#(.*)')
    
    return df_entrylist

In [116]:
get_entry_list(stub)

,CarNum,DriverName,CoDriverName,Team,Car,Class,Meta_0,Meta_1,carNum
0,NaN,Bernardi Florian,Bellotto Victor,Renault Clio RSR Rally 5,[FK-845-ZT],NaN,NaN,Course car,NaN
1,NaN,Henry Patrick,Lombard Magali,Renault Mégane IV RS,NaN,NaN,NaN,Course car,NaN
2,#8,Tänak Ott,Järveoja Martin,Hyundai i20 Coupe WRCHyundai Shell Mobis WRT,[ALZ WR 41],RC1,M,NaN,8
3,#11,Neuville Thierry,Gilsoul Nicolas,Hyundai i20 Coupe WRCHyundai Shell Mobis WRT,[ALZ WR 42],RC1,M,NaN,11
4,#17,Ogier Sébastien,Ingrassia Julien,Toyota Yaris WRCToyota Gazoo Racing WRT,[SP 1015],RC1,M,NaN,17
...,...,...,...,...,...,...,...,...,...
85,#96,Desbordes Richard,Desbordes Steven,Citroën DS3 R1,NaN,RC5,NaN,NaN,96
86,#97,Villy Ronald,Villy Jonathan,Citroën DS3 R1,NaN,RC5,NaN,NaN,97
87,#98,Berard Christophe,Bernabo Christophe,Ford Fiesta R2T National,NaN,RC5,NaN,NaN,98
88,#99,Lartillier Thierry,Lemoine Christophe,Renault Twingo RS R1,[DZ 133 XH],RC5,NaN,NaN,99


## Rebasers

Utils for rebasing

In [117]:
def _rebaseTimes(times, bib=None, basetimes=None):
    ''' Rebase times relative to specified driver. '''
    #Should we rebase against entryId, so need to lool that up. In which case, leave index as entryId
    if bib is None and basetimes is None: return times
    #bibid = codes[codes['Code']==bib].index.tolist()[0]
    if bib is not None:
        return times - times.loc[bib]
    if times is not None:
        return times - basetimes
    return times

## `EWRC` Class
Create a class to that can be used to gran all the results for a particular rally, as required.

We fudge the definition of class functions so that we can separately define and functions in a standalione way. This is probably *not good practice*...!

In [2]:
class EWRC:
    """Class for eWRC data for a particular rally."""

    def __init__(self, stub, live=False):
        self.stub = stub
        self.live = live
        
        self.stage_result_links = None
        
        self.df_rally_overall = None
        
        self.df_allInOne = None #we don't actually use this?
        self.df_overall = None
        self.df_stages = None
        self.df_overall_pos = None
        self.df_stages_pos = None
        
        self.df_overall_rebased_to_leader = None
        self.df_stages_rebased_to_overall_leader = None
        self.df_stages_rebased_to_stage_winner = None
        
        self.event_dist = None
        self.df_itinerary_leg_totals = None
        self.df_itinerary = None
        self.df_full_itinerary =None

        self.stage_distances_all = None
        self.stage_distances = None
        
        self.df_entry_list = None
        self.rally_classes = None
        
        self.entryFromCar = None
        self.carFromEntry = None
 
        self.df_stage_result = pd.DataFrame(columns=stage_result_cols)
        self.df_stage_overall = pd.DataFrame(columns=stage_overall_cols)
        self.df_stage_retirements = pd.DataFrame(columns=retirement_cols+retirement_extra_cols)
        self.df_stage_penalties = pd.DataFrame(columns=penalty_cols+penalty_extra_cols)

    def df_inclass_cars(self, _df, rally_class='all', typ='entryId'):
        """Get cars in particular class."""
        if rally_class != 'all':
            _df = _df[_df.index.isin(self.carsInClass(rally_class, typ=typ))]
        return _df

    def carsInClass(self, qclass, typ='carNum'):
        #Can't we also pass a dict of key/vals to the widget?
        #Omit car 0
        df_entry_list = self.get_entry_list()
        if qclass.lower()=='all':
            return df_entry_list[df_entry_list['CarNum']!='#0']['carNum'].dropna().to_list()
        _cars = df_entry_list[(df_entry_list['CarNum']!='#0') & (df_entry_list['Class']==qclass)]['carNum'].to_list()
        if typ=='entryId':
            _cars = [self.entryFromCar[c] for c in self.entryFromCar if c in _cars]
        return _cars

    def stages_class_winners(self, rally_class='all'):
        """Return stage winners for a specified class."""
        _class_stage_winners = self.df_inclass_cars(self.df_stages_pos,
                                                    rally_class=rally_class).idxmin()
        return _class_stage_winners
    
    def get_class_rebased_times(self, rally_class='all', typ='stagewinner'):
        """
        Get times rebased relative to class.
        Rebaser can be either class stage winner or class stage overall.
        """
        # TO DO  - not yet implemented for class overall
        self.get_stage_times()
        _stage_times = self.df_inclass_cars(self.df_stages, rally_class=rally_class)
        df_stages_rebased_to_stage_winner = _stage_times.apply(_rebaseTimes,
                                                               basetimes=_stage_times.min(), axis=1)
        return df_stages_rebased_to_stage_winner

    def set_rebased_times(self):
        if self.df_stages_rebased_to_overall_leader is None \
                or self.df_stages_rebased_to_stage_winner is None \
                or self.df_stages_rebased_to_stage_winner is None:
            #print('setting rebased times...')
            self.get_stage_times()
            leaderStagetimes = self.df_stages.iloc[0]
            self.df_stages_rebased_to_overall_leader = self.df_stages.apply(_rebaseTimes,
                                                                            basetimes=leaderStagetimes, axis=1)
            #Now rebase to the stage winner
            self.df_stages_rebased_to_stage_winner = self.df_stages_rebased_to_overall_leader.apply(_rebaseTimes, basetimes=self.df_stages_rebased_to_overall_leader.min(), axis=1)

            leaderTimes = self.df_overall.min()
            self.df_overall_rebased_to_leader = self.df_overall.apply(_rebaseTimes,
                                                                      basetimes=leaderTimes, axis=1)

    def _set_car_entry_lookups(self, df, force=False):
        """Look-up dicts between car number and entry."""
        if force or not self.carFromEntry or not self.entryFromCar:
            self.carFromEntry = df['carNum'].to_dict()
            self.entryFromCar =  {v:k for (k, v) in self.carFromEntry.items()}
    
    def get_final(self):
        if self.df_rally_overall is None:
            self.df_rally_overall = get_final(self.stub)
            self._set_car_entry_lookups(self.df_rally_overall)
        return self.df_rally_overall
        
    def get_stage_times(self):
        if self.live or self.df_overall is None or self.df_stages is None or self.df_overall_pos is None:
            self.df_allInOne, self.df_overall, self.df_stages, \
                self.df_overall_pos, self.df_stages_pos = get_stage_times(self.stub)
            self._set_car_entry_lookups(self.df_allInOne)
        return self.df_allInOne, self.df_overall, self.df_stages, self.df_overall_pos
    
    def get_itinerary(self):
        if self.live or self.event_dist is None or self.df_itinerary_leg_totals is None \
            or self.df_itinerary is None or self.df_full_itinerary is None:
                self.event_dist, self.df_itinerary_leg_totals, \
                    self.df_itinerary, self.df_full_itinerary_df = get_itinerary(self.stub)

        _stage_distances = self.df_itinerary['Distance'][~self.df_itinerary['Time'].str.contains('cancelled')]
        # Stage distances do not include cancelled stages
        # The following is the correct stage index
        #_stage_distances.index = [int(i.lstrip('SS')) for i in _stage_distances.index]
        #As a hack, to cope with cancelled stages, reindex
        _stage_distances.reset_index(drop=True, inplace=True)
        _stage_distances.index += 1 
        self.stage_distances = _stage_distances
        self.stage_distances_all = self.df_itinerary['Distance']
    
        return self.event_dist, self.df_itinerary_leg_totals, \
                self.df_itinerary, self.df_full_itinerary
    
    def get_entry_list(self):
        if self.df_entry_list is None:
            self.df_entry_list = get_entry_list(self.stub)
            
        #A list of classes could be useful, so grab it while we can
        self.rally_classes = self.df_entry_list['Class'].dropna().unique()
        
        return self.df_entry_list
    
    def get_stage_result_links(self):
        if self.stage_result_links is None:
            self.stage_result_links = get_stage_result_links(self.stub)
        return self.stage_result_links
    
    def get_stage_results(self, stage=None):
        #for now, just return what we have with stage as None
        if stage is None:
            return self.df_stage_result, self.df_stage_overall, \
                    self.df_stage_retirements, self.df_stage_penalties
        # Could maybe change that to get everything?
        stages = stage if isinstance(stage,list) else [stage]
        if stages:
            links = self.get_stage_result_links()
            if 'all' in stages:
                print('all')
                stages = [k for k in links.keys() if 'leg' not in k]
            elif 'final' in stages or 'last' in stages:
                stages = [k for k in links.keys() if 'leg' not in k][-1]
            for stage in stages:
                if self.live or (stage not in self.df_stage_result['Stage'].unique() and stage in links):
                    df_stage_result, df_stage_overall, df_stage_retirements, \
                        df_stage_penalties = get_stage_results(links[stage])
                    self.df_stage_result = self.df_stage_result.append(df_stage_result, sort=False).reset_index(drop=True)
                    self.df_stage_overall = self.df_stage_overall.append(df_stage_overall, sort=False).reset_index(drop=True)
                    self.df_stage_retirements = self.df_stage_retirements.append(df_stage_retirements, sort=False).reset_index(drop=True)
                    self.df_stage_penalties = self.df_stage_penalties.append(df_stage_penalties, sort=False).reset_index(drop=True)
        
        if stages:
            return self.df_stage_result[self.df_stage_result['Stage'].isin(stages)], \
                    self.df_stage_overall[self.df_stage_overall['Stage'].isin(stages)], \
                    self.df_stage_retirements[self.df_stage_retirements['Stage'].isin(stages)], \
                    self.df_stage_penalties[self.df_stage_penalties['Stage'].isin(stages)]
        
        self._set_car_entry_lookups(self.df_stage_result)
        
        return self.df_stage_result, self.df_stage_overall, \
                self.df_stage_retirements, self.df_stage_penalties

In [119]:
ewrc=EWRC(rally_stub)

In [120]:
#df_stage_result, df_stage_overall, df_stage_retirements, df_stage_penalties
ewrc.get_stage_results()

(Empty DataFrame
 Columns: [Pos, CarNum, Desc, Class, Time, GapDiff, Speedkm, Stage, StageName, StageDist, Gap, Diff, Speed, Dist, entryId, model, navigator, PosNum]
 Index: [], Empty DataFrame
 Columns: [PosChange, CarNum, Desc, Class, Time, GapDiff, Speedkm, Stage, StageName, StageDist, Pos, Change, Gap, Diff, Speed, Dist]
 Index: [], Empty DataFrame
 Columns: [CarNum, driverNav, Model, Status, Driver, CoDriver, Stage]
 Index: [], Empty DataFrame
 Columns: [CarNum, driverNav, Model, PenReason, Driver, CoDriver, Stage, Time, Reason]
 Index: [])

In [148]:
ewrc.get_stage_results('SS1')[0]

,Pos,CarNum,Desc,Class,Time,GapDiff,Speedkm,Stage,StageName,StageDist,Gap,Diff,Speed,Dist,entryId,model,navigator,PosNum
91,1.,33,Evans Elfyn - Martin ScottFord Fiesta WRC,RC1M,10:20.6,NaN,102.1,SS1,Bavella 1,17.6,0.0,0.0,102.1,,Ford Fiesta WRC,Evans Elfyn,Martin Scott,1
92,2.,11,Neuville Thierry - Gilsoul N.Hyundai i20 Coupe...,RC1M,10:23.5,+2.9+2.9,101.60.16,SS1,Bavella 1,17.6,2.9,2.9,101.6,0.16,Hyundai i20 Coupe WRC,Neuville Thierry,Gilsoul N.,2
93,3.,6,Sordo Dani - del Barrio CarlosHyundai i20 Coup...,RC1M,10:23.7,+3.1+0.2,101.60.18,SS1,Bavella 1,17.6,3.1,0.2,101.6,0.18,Hyundai i20 Coupe WRC,Sordo Dani,del Barrio Carlos,3
94,4.,8,Tänak Ott - Järveoja MartinToyota Yaris WRC,RC1M,10:23.8,+3.2+0.1,101.60.18,SS1,Bavella 1,17.6,3.2,0.1,101.6,0.18,Toyota Yaris WRC,Tänak Ott,Järveoja Martin,4
95,5.,3,Suninen Teemu - Salminen MarkoFord Fiesta WRC,RC1M,10:25.4,+4.8+1.6,101.30.27,SS1,Bavella 1,17.6,4.8,1.6,101.3,0.27,Ford Fiesta WRC,Suninen Teemu,Salminen Marko,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,89.,108,d'Ulivo Nicolas - Paolini A.Renault Twingo RS R1,RC5,14:20.2,+3:59.6+14.7,73.713.6,SS1,Bavella 1,17.6,239.6,14.7,73.7,13.6,Renault Twingo RS R1,d'Ulivo Nicolas,Paolini A.,89
180,90.,63,Bousquet Thierry - Defrance E.Renault Clio RS R3T,RC3,14:24.8,+4:04.2+4.6,73.313.9,SS1,Bavella 1,17.6,244.2,4.6,73.3,13.9,Renault Clio RS R3T,Bousquet Thierry,Defrance E.,90
181,91.,109,Dutreuil Guy - Mallon LaurentCitroën DS3 R1,RC5,14:30.5,+4:09.9+5.7,72.814.2,SS1,Bavella 1,17.6,249.9,5.7,72.8,14.2,Citroën DS3 R1,Dutreuil Guy,Mallon Laurent,91
182,92.,64,Kemp Pierre - Kemp Jean MarcRenault Clio R3,RC3,14:36.0,+4:15.4+5.5,72.314.5,SS1,Bavella 1,17.6,255.4,5.5,72.3,14.5,Renault Clio R3,Kemp Pierre,Kemp Jean Marc,92


In [142]:
ewrc.get_stage_results('SS1')[1]

,PosChange,CarNum,Desc,Class,Time,GapDiff,Speedkm,Stage,StageName,StageDist,Pos,Change,Gap,Diff,Speed,Dist
91,1.,33,Evans Elfyn - Martin ScottFord Fiesta WRC,RC1M,10:20.6,NaN,102.1,SS1,Bavella 1,17.6,1,,0.0,0.0,102.1,
92,2.,11,Neuville T. - Gilsoul N.Hyundai i20 Coupe WRC,RC1M,10:23.5,+2.9+2.9,101.60.16,SS1,Bavella 1,17.6,2,,2.9,2.9,101.6,0.16
93,3.,6,Sordo Dani - del Barrio C.Hyundai i20 Coupe WRC,RC1M,10:23.7,+3.1+0.2,101.60.18,SS1,Bavella 1,17.6,3,,3.1,0.2,101.6,0.18
94,4.,8,Tänak Ott - Järveoja M.Toyota Yaris WRC,RC1M,10:23.8,+3.2+0.1,101.60.18,SS1,Bavella 1,17.6,4,,3.2,0.1,101.6,0.18
95,5.,3,Suninen Teemu - Salminen M.Ford Fiesta WRC,RC1M,10:25.4,+4.8+1.6,101.30.27,SS1,Bavella 1,17.6,5,,4.8,1.6,101.3,0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,89.,109,Dutreuil Guy - Mallon L.Citroën DS3 R1,RC5,14:30.5,+4:09.9+5.7,72.814.2,SS1,Bavella 1,17.6,89,,249.9,5.7,72.8,14.2
180,90.,60,Coufranc M. - Caddeo A.Renault Clio R3,RC3,14:35.20:40,+4:14.6+4.7,75.914.5,SS1,Bavella 1,17.6,90,,254.6,4.7,75.9,14.5
181,91.,64,Kemp Pierre - Kemp J.Renault Clio R3,RC3,14:56.00:20,+4:35.4+20.8,72.315.7,SS1,Bavella 1,17.6,91,,275.4,20.8,72.3,15.7
182,92.,79,Torn Ken - Pannas KauriFord Fiesta R2T19,RC4,17:11.65:00,+6:51.0+2:15.6,86.623.4,SS1,Bavella 1,17.6,92,,411.0,135.6,86.6,23.4


In [122]:
ewrc.set_rebased_times()
display(ewrc.df_stages_rebased_to_overall_leader)

,1,2,3,4
entryId,,,,
/entryinfo/59972-rallye-automobile-de-monte-carlo-2020/2465687/,0.0,0.0,0.0,0.0
/entryinfo/59972-rallye-automobile-de-monte-carlo-2020/2465681/,4.5,-29.9,20.0,8.8
/entryinfo/59972-rallye-automobile-de-monte-carlo-2020/2465684/,-1.9,-4.4,8.4,3.4
/entryinfo/59972-rallye-automobile-de-monte-carlo-2020/2465689/,9.0,16.6,2.1,9.2
/entryinfo/59972-rallye-automobile-de-monte-carlo-2020/2487703/,19.7,22.7,13.6,12.0
...,...,...,...,...
/entryinfo/59972-rallye-automobile-de-monte-carlo-2020/2498718/,633.8,659.4,151.9,NaN
/entryinfo/59972-rallye-automobile-de-monte-carlo-2020/2465680/,-0.1,-0.2,5.2,NaN
/entryinfo/59972-rallye-automobile-de-monte-carlo-2020/2487697/,159.6,201.7,NaN,NaN


In [143]:
ewrc.get_stage_results('SS1')[2]

,CarNum,driverNav,Model,Status,Driver,CoDriver,Stage
2,50,Giorgi Fabrice - Marchini J.,Škoda Fabia R5,Not allowed to start,Giorgi Fabrice,Marchini J.,SS1
3,92,Murati J. - Guintini P.,Ford Fiesta R2T,Engine,Murati J.,Guintini P.,SS1


In [123]:
ewrc.get_stage_result_links()

{'SS1': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241659',
 'SS2': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241660',
 'SS3': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241661',
 'SS4': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241662',
 'SS5': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241663',
 'SS6': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241664',
 'SS7': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241665',
 'SS8': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241666',
 'SS2. leg': '/leg/59972-rallye-automobile-de-monte-carlo-2020/?leg=2',
 'SS9': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241667',
 'SS10': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241668',
 'SS11': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241669',
 'SS12': '/results/59972-rallye-automobile-de-monte-carlo-2020/?s=241670',
 'SS3. leg': '/leg/59972-rallye-autom

In [144]:
ewrc.get_stage_results('SS1')[3]

,CarNum,driverNav,Model,PenReason,Driver,CoDriver,Stage,Time,Reason
1,79,Torn Ken - Pannas Kauri,Ford Fiesta R2T19,5:00 Change of engine,Torn Ken,Pannas Kauri,SS1,5:00,Change of engine
2,60,Coufranc Michaël - Caddeo A.,Renault Clio R3,0:40 Late (4 min) at TC,Coufranc Michaël,Caddeo A.,SS1,0:40,Late (4 min) at TC
3,64,Kemp Pierre - Kemp Jean Marc,Renault Clio R3,0:20 Late (2 min) at TC,Kemp Pierre,Kemp Jean Marc,SS1,0:20,Late (2 min) at TC
4,96,Leca Nicolas - Orts J.,Peugeot 208 R2,0:10 Late (1 min) at TC,Leca Nicolas,Orts J.,SS1,0:10,Late (1 min) at TC


In [135]:
ewrc.get_stage_times()[0]

,driverNav,driver,navigator,carNum,carModel,retired,Pos,stagetimes,overalltimes,positions,penalties
entryId,,,,,,,,,,,
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121689/,Neuville Thierry - Gilsoul Nicolas,Neuville Thierry,Gilsoul Nicolas,11,Hyundai i20 Coupe WRC,False,1.0,"[10:23.5, 14:27.7, 10:06.3, 10:24.0, 14:25.6, ...","[10:23.5, 24:51.2, 34:57.5, 45:21.5, 59:47.1, ...","[2, 4, 3, 3, 3, 3, 3, 3, 3, 3, 2, 1, 2, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121688/,Ogier Sébastien - Ingrassia Julien,Ogier Sébastien,Ingrassia Julien,1,Citroën C3 WRC,False,2.0,"[10:33.1, 14:36.7, 10:08.4, 10:22.1, 14:29.7, ...","[10:33.1, 25:09.8, 35:18.2, 45:40.3, 1:00:10.0...","[8, 8, 8, 7, 6, 6, 6, 6, 5, 5, 4, 3, 3, 2]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121708/,Evans Elfyn - Martin Scott,Evans Elfyn,Martin Scott,33,Ford Fiesta WRC,False,3.0,"[10:20.6, 14:28.8, 10:05.9, 10:17.5, 14:23.2, ...","[10:20.6, 24:49.4, 34:55.3, 45:12.8, 59:36.0, ...","[1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 3]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121707/,Sordo Dani - del Barrio Carlos,Sordo Dani,del Barrio Carlos,6,Hyundai i20 Coupe WRC,False,4.0,"[10:23.7, 14:26.5, 10:11.4, 10:25.7, 14:30.3, ...","[10:23.7, 24:50.2, 35:01.6, 45:27.3, 59:57.6, ...","[3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121700/,Suninen Teemu - Salminen Marko,Suninen Teemu,Salminen Marko,3,Ford Fiesta WRC,False,5.0,"[10:25.4, 14:34.4, 10:10.7, 10:23.1, 14:30.5, ...","[10:25.4, 24:59.8, 35:10.5, 45:33.6, 1:00:04.1...","[5, 6, 5, 5, 5, 5, 5, 5, 6, 6, 5, 5, 5, 5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2181013/,Chiaroni Pascal - Damerval Stéphane,Chiaroni Pascal,Damerval Stéphane,59,Renault Clio R3,True,NaN,"[12:56.2, 17:16.7]","[12:56.2, 30:12.9]","[73, 60]","[0, 0]"
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2180990/,Johnston Sean - Kihurani Alexander,Johnston Sean,Kihurani Alexander,77,Ford Fiesta R2T19,True,NaN,[12:05.6],[12:05.6],[41],[0]
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2181038/,Donati Xavier - Ciavaldini France,Donati Xavier,Ciavaldini France,102,Peugeot 208 R2,True,NaN,[12:54.2],[12:54.2],[72],[0]


In [136]:
ewrc.get_stage_times()[1]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
entryId,,,,,,,,,,,,,,
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121689/,623.5,1491.2,2097.5,2721.5,3587.1,4189.4,5143.2,5621.0,7413.5,8368.0,8845.6,10610.0,11573.2,12179.0
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121688/,633.1,1509.8,2118.2,2740.3,3610.0,4215.9,5176.7,5659.6,7446.0,8404.6,8887.3,10654.8,11612.8,12219.3
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121708/,620.6,1489.4,2095.3,2712.8,3576.0,4179.6,5130.8,5610.3,7398.6,8353.0,8834.1,10614.5,11561.7,12245.6
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121707/,623.7,1490.2,2101.6,2727.3,3597.6,4205.7,5165.3,5644.6,7429.6,8392.3,8872.9,10659.9,11640.9,12257.4
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121700/,625.4,1499.8,2110.5,2733.6,3604.1,4210.5,5174.6,5655.2,7470.0,8432.0,8911.0,10702.1,11659.7,12263.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2181013/,776.2,1812.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2180990/,725.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2181038/,774.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
ewrc.get_stage_times()[2]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
entryId,,,,,,,,,,,,,,
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121689/,623.5,867.7,606.3,624.0,865.6,602.3,953.8,477.8,1792.5,954.5,477.6,1764.4,963.2,605.8
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121688/,633.1,876.7,608.4,622.1,869.7,605.9,960.8,482.9,1786.4,958.6,482.7,1767.5,958.0,606.5
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121708/,620.6,868.8,605.9,617.5,863.2,603.6,951.2,479.5,1788.3,954.4,481.1,1780.4,947.2,683.9
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121707/,623.7,866.5,611.4,625.7,870.3,608.1,959.6,479.3,1785.0,962.7,480.6,1787.0,981.0,616.5
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121700/,625.4,874.4,610.7,623.1,870.5,606.4,964.1,480.6,1814.8,962.0,479.0,1791.1,957.6,603.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2181013/,776.2,1036.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2180990/,725.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2181038/,774.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
ewrc.get_stage_times()[3]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
entryId,,,,,,,,,,,,,,
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121689/,2.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,1.0,2.0,1.0
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121688/,8.0,8.0,8.0,7.0,6.0,6.0,6.0,6.0,5.0,5.0,4.0,3.0,3.0,2.0
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121708/,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,3.0
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121707/,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121700/,5.0,6.0,5.0,5.0,5.0,5.0,5.0,5.0,6.0,6.0,5.0,5.0,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2181013/,73.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2180990/,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2181038/,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
ewrc.get_itinerary()

('304.28 km',   Leg   distance
 0   1   42.96 km
 1   2  122.58 km
 2   3   75.20 km
 3   4   63.54 km,                                               Name  distance    Date   Time  \
 Stage                                                                         
 SS1                            Malijai - Puimichel  17.47 km  23. 1.  20:38   
 SS2                              Bayons - Bréziers  25.49 km  23. 1.  22:26   
 SS3                           Curbans - Venterol 1  20.02 km  24. 1.  08:36   
 SS4       St-Clément-sur-Durance - Freissinières 1  20.68 km  24. 1.  09:56   
 SS5                 Avançon - Notre-Dame-du-Laus 1  20.59 km  24. 1.  11:21   
 SS6                           Curbans - Venterol 2  20.02 km  24. 1.  13:54   
 SS7       St-Clément-sur-Durance - Freissinières 2  20.68 km  24. 1.  15:14   
 SS8                 Avançon - Notre-Dame-du-Laus 2  20.59 km  24. 1.  16:39   
 SS9        St-Léger-les-Mélèzes - La Bâtie-Neuve 1  16.87 km  25. 1.  09:38   
 SS10            

In [126]:
ewrc.get_entry_list()

,CarNum,DriverName,CoDriverName,Team,Car,Class,Meta_0,Meta_1,carNum
0,NaN,Bernardi Florian,Bellotto Victor,Renault Clio RSR Rally 5,[FK-845-ZT],NaN,NaN,Course car,NaN
1,NaN,Henry Patrick,Lombard Magali,Renault Mégane IV RS,NaN,NaN,NaN,Course car,NaN
2,#8,Tänak Ott,Järveoja Martin,Hyundai i20 Coupe WRCHyundai Shell Mobis WRT,[ALZ WR 41],RC1,M,NaN,8
3,#11,Neuville Thierry,Gilsoul Nicolas,Hyundai i20 Coupe WRCHyundai Shell Mobis WRT,[ALZ WR 42],RC1,M,NaN,11
4,#17,Ogier Sébastien,Ingrassia Julien,Toyota Yaris WRCToyota Gazoo Racing WRT,[SP 1015],RC1,M,NaN,17
...,...,...,...,...,...,...,...,...,...
85,#96,Desbordes Richard,Desbordes Steven,Citroën DS3 R1,NaN,RC5,NaN,NaN,96
86,#97,Villy Ronald,Villy Jonathan,Citroën DS3 R1,NaN,RC5,NaN,NaN,97
87,#98,Berard Christophe,Bernabo Christophe,Ford Fiesta R2T National,NaN,RC5,NaN,NaN,98
88,#99,Lartillier Thierry,Lemoine Christophe,Renault Twingo RS R1,[DZ 133 XH],RC5,NaN,NaN,99


In [127]:
ewrc=EWRC('54464-corsica-linea-tour-de-corse-2019')

In [128]:
ewrc.get_entry_list()

,CarNum,DriverName,CoDriverName,Team,Car,Class,Meta_0,Meta_1,carNum
0,NaN,Bengué Alexandre,Chauffray Thomas,Citroën C3 R5,[chassis 2],NaN,NaN,Course car,NaN
1,#8,Tänak Ott,Järveoja Martin,Toyota Yaris WRCToyota Gazoo Racing WRT,[SP 1010],RC1,M,NaN,8
2,#11,Neuville Thierry,Gilsoul Nicolas,Hyundai i20 Coupe WRCHyundai Shell Mobis WRT,[ALZ WR 37],RC1,M,NaN,11
3,#1,Ogier Sébastien,Ingrassia Julien,Citroën C3 WRCCitroën Total WRT,[FB-061-PF],RC1,M,NaN,1
4,#5,Meeke Kris,Marshall Sebastian,Toyota Yaris WRCToyota Gazoo Racing WRT,[SP 1011],RC1,M,NaN,5
...,...,...,...,...,...,...,...,...,...
91,#106,Lemaire Martin,Barboni Philippe,Citroën C2 R2 Max,NaN,RC4,NaN,NaN,106
92,#107,Bracconi Emmanuel,Saoletti Florian,Renault Twingo RS R1,NaN,RC5,NaN,NaN,107
93,#108,d'Ulivo Nicolas,Paolini Angélique,Renault Twingo RS R1,NaN,RC5,NaN,NaN,108
94,#109,Dutreuil Guy,Mallon Laurent,Citroën DS3 R1,NaN,RC5,NaN,NaN,109


In [130]:
ewrc.get_stage_results('SS2')[0]

,Pos,CarNum,Desc,Class,Time,GapDiff,Speedkm,Stage,StageName,StageDist,Gap,Diff,Speed,Dist,entryId,model,navigator,PosNum
0,1.,5,Meeke Kris - Marshall SebastianToyota Yaris WRC,RC1M,14:23.6,NaN,108.1,SS2,Valinco 1,25.94,0.0,0.0,108.1,,Toyota Yaris WRC,Meeke Kris,Marshall Sebastian,1
1,2.,8,Tänak Ott - Järveoja MartinToyota Yaris WRC,RC1M,14:25.0,+1.4+1.4,108.00.05,SS2,Valinco 1,25.94,1.4,1.4,108.0,0.05,Toyota Yaris WRC,Tänak Ott,Järveoja Martin,2
2,3.,6,Sordo Dani - del Barrio CarlosHyundai i20 Coup...,RC1M,14:26.5,+2.9+1.5,107.80.11,SS2,Valinco 1,25.94,2.9,1.5,107.8,0.11,Hyundai i20 Coupe WRC,Sordo Dani,del Barrio Carlos,3
3,4.,11,Neuville Thierry - Gilsoul N.Hyundai i20 Coupe...,RC1M,14:27.7,+4.1+1.2,107.60.16,SS2,Valinco 1,25.94,4.1,1.2,107.6,0.16,Hyundai i20 Coupe WRC,Neuville Thierry,Gilsoul N.,4
4,5.,33,Evans Elfyn - Martin ScottFord Fiesta WRC,RC1M,14:28.8,+5.2+1.1,107.50.20,SS2,Valinco 1,25.94,5.2,1.1,107.5,0.20,Ford Fiesta WRC,Evans Elfyn,Martin Scott,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,87.,109,Dutreuil Guy - Mallon LaurentCitroën DS3 R1,RC5,20:01.3,+5:37.7+6.1,77.713.0,SS2,Valinco 1,25.94,337.7,6.1,77.7,13.0,Citroën DS3 R1,Dutreuil Guy,Mallon Laurent,87
87,88.,61,Sassi Alberto - Cangini FabioAbarth 124 Rally RGT,RGT,21:27.1,+7:03.5+1:25.8,72.616.3,SS2,Valinco 1,25.94,423.5,85.8,72.6,16.3,Abarth 124 Rally RGT,Sassi Alberto,Cangini Fabio,88
88,89.,84,Fraymouth F. - Felicelli R.Peugeot 208 R2,RC4,22:18.8,+7:55.2+51.7,69.818.3,SS2,Valinco 1,25.94,475.2,51.7,69.8,18.3,Peugeot 208 R2,Fraymouth F.,Felicelli R.,89
89,90.,43,Simonetti Robert - Simonetti C.Citroën DS3 WRC...,RC1,23:42.0,+9:18.4+1:23.2,65.721.5,SS2,Valinco 1,25.94,558.4,83.2,65.7,21.5,Citroën DS3 WRC,Simonetti Robert,Simonetti C.,90


In [131]:
ewrc.get_stage_results('SS2')[1]

,PosChange,CarNum,Desc,Class,Time,GapDiff,Speedkm,Stage,StageName,StageDist,Pos,Change,Gap,Diff,Speed,Dist
0,1. +3,8,Tänak Ott - Järveoja M.Toyota Yaris WRC,RC1M,24:48.8,NaN,105.3,SS2,Valinco 1,25.94,1,+3,0.0,0.0,105.3,
1,2. −1,33,Evans Elfyn - Martin ScottFord Fiesta WRC,RC1M,24:49.4,+0.6+0.6,105.20.01,SS2,Valinco 1,25.94,2,−1,0.6,0.6,105.2,0.01
2,3.,6,Sordo Dani - del Barrio C.Hyundai i20 Coupe WRC,RC1M,24:50.2,+1.4+0.8,105.20.03,SS2,Valinco 1,25.94,3,,1.4,0.8,105.2,0.03
3,4. −2,11,Neuville T. - Gilsoul N.Hyundai i20 Coupe WRC,RC1M,24:51.2,+2.4+1.0,105.10.06,SS2,Valinco 1,25.94,4,−2,2.4,1.0,105.1,0.06
4,5. +2,4,Lappi Esapekka - Ferm JanneCitroën C3 WRC,RC1M,24:58.2,+9.4+7.0,104.60.22,SS2,Valinco 1,25.94,5,+2,9.4,7.0,104.6,0.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,87. −47,84,Fraymouth F. - Felicelli R.Peugeot 208 R2,RC4,34:24.0,+9:35.2+8.3,75.913.2,SS2,Valinco 1,25.94,87,−47,575.2,8.3,75.9,13.2
87,88. +1,109,Dutreuil Guy - Mallon L.Citroën DS3 R1,RC5,34:31.8,+9:43.0+7.8,75.713.4,SS2,Valinco 1,25.94,88,+1,583.0,7.8,75.7,13.4
88,89. +2,64,Kemp Pierre - Kemp J.Renault Clio R3,RC3,34:51.20:20,+10:02.4+19.4,75.713.8,SS2,Valinco 1,25.94,89,+2,602.4,19.4,75.7,13.8
89,90. +3,43,Simonetti R. - Simonetti C.Citroën DS3 WRC [SR],RC1,42:28.1,+17:39.3+7:36.9,61.524.3,SS2,Valinco 1,25.94,90,+3,1059.3,456.9,61.5,24.3


In [132]:
ewrc.get_stage_results('SS2')[2]

,CarNum,driverNav,Model,Status,Driver,CoDriver,Stage
0,77,Johnston Sean - Kihurani A.,Ford Fiesta R2T19,Accident,Johnston Sean,Kihurani A.,SS2
1,102,Donati Xavier - Ciavaldini F.,Peugeot 208 R2,Mechanical,Donati Xavier,Ciavaldini F.,SS2


In [133]:
ewrc.get_stage_results('SS2')[3]

,CarNum,driverNav,Model,PenReason,Driver,CoDriver,Stage,Time,Reason
0,52,Göttig S. - Solbach-Schmidt N.,Škoda Fabia R5,0:10 Late (1 min) at TC,Göttig S.,Solbach-Schmidt N.,SS2,0:10,Late (1 min) at TC
